# [Mastermind](https://fr.wikipedia.org/wiki/Mastermind)
## Objet du problème
On se propose de faire jouer à l'ordinateur le rôle du décodeur dans le jeu Mastermind. L'algorithme utilisé, dû à [Donald Knuth](https://fr.wikipedia.org/wiki/Donald_Knuth), est nommé *five-guess* car il permet de trouver le code caché en au plus cinq propositions (cinq motifs proposés).

On donne ensuite un autre algorithme (D. L. Greenwell), pour le mastermind classique ($p=4$ positions et $n=6$ couleurs), qui consiste à faire 6 propositions d'un coup, avec toujours les 6 mêmes motifs, pour ensuite déduire le code caché des indications associées à ces 6 propositions.

## Mise en place des données du jeu
Les couleurs étant numérotées de $0$ à $n-1$, un code ou un un motif est un élément de $\{0,\ldots,n-1\}^p$ ; une indication est un couple (nombre de bien placés, nombre de mal placés).  
Dans le but d'afficher les codes, motifs et indications, on les représente par des jolies chaines de caractères colorées en utilisant l'unicode.  

In [1]:
p, n = 4, 6

colors = '🔴🟢🔵🟡⚫⚪🟣🟠' # n supposé <= 8

def repr(t):
    if len(t) == p:
        s = ''
        for i in t: s += colors[i]
        return s
    else:
        return '● ' * t[0] + '⚬ ' * t[1] + '  ' * (p - t[0] - t[1]) 

`totalCodes` $=\{0,\ldots,n-1\}^p$ est l'ensemble de tous le codes (ou motifs) possibles.   
`hints` est la liste des indications possibles.  
`getHint`$(c,m)$ calcule l'indication $h$ associée à un motif $m$, le code $c$ étant connu

In [2]:
from itertools import product
totalCodes = set(product(*(range(n) for _ in range(p))))
hints = [(w, b) for w in range(p + 1) for b in range(p + 1 - w) if (w,b) != (p - 1, 1)]

def getHint(code, pattern):
    well = len([i for i in range(p) if code[i] == pattern[i]])
    def N(t, j):
        return len([i for i in range(p) if t[i] == j])
    bad = sum([min(N(code,j), N(pattern,j)) for j in range(n)]) - well
    return well, bad

On calcule touts les `getHint`$(c,m)$ et on les stocke dans un dictionnaire `totalHints`.

In [3]:
# 1/2 mn pour p, n = 4, 6
totalHints = dict()
for code in totalCodes:
    for pattern in totalCodes:
        totalHints[(code, pattern)] = getHint(code, pattern)
print(f'{n**p}^2 = {n**(2*p)} calculs')

1296^2 = 1679616 calculs


La fonction `decreaseCodes` prend en entrée
- un ensemble $C$ de codes (une partie de `totalCodes`) qui contient le code caché ;
- un motif $m$ et son indication associée $h$.

Elle renvoie la partie $C'$ de $C$ formée des codes de $C$ qui, face au motif $m$, fournirait la même indication $h$.  
Ainsi, $C'$ contient encore le code caché et possède moins d'éléments que $C$.

In [4]:
def decreaseCodes(codes, pattern, hint):

    return {code for code in codes if totalHints[(code, pattern)] == hint}

## L'algorithme five-guess
La fonction `nextPattern`$(C)$ est la fonction principale de l'algorithme. Elle prend en entrée un ensemble $C$ de codes qui contient le code caché et renvoie le motif $m$ qui, en un certain sens, fera le plus décroitre $C$. Remarque pas très maline : si on connaissait à l'avance l'indication $h$, on choisirait le motif pour lequel le cardinal de `decreaseCodes`$(C,m,h)$ soit minimum. En fait, on choisit $m$ pour que max $\{$ card $($`decreaseCodes`$(C,m,h)), h\in$ `hints`$\}$ soit minimum.

In [5]:
def nextPattern(codes):

    if len(codes) == 1: return codes.pop()
    wMin = len(codes)
    for pattern in totalCodes:
        W = 0
        for hint in hints:
            w = len(decreaseCodes(codes, pattern, hint))
            W = max(W, w)
        if W < wMin:
            wMin = W
            patternMin = pattern
    return patternMin

Comme le calcul est un peu long (15 s pour $p,n=4,6$), on calcule une fois pour toutes le premier motif à proposer 

In [6]:
initPattern = nextPattern(totalCodes)
repr(initPattern)

'⚪🔴🔴⚪'

La fonction suivante simule l'exécution de l'algorithme pour un code "caché" donné

In [7]:
def simulateKnuth(secretCode, verbose = True):
    
    if verbose:
        print(f'secret : {repr(secretCode)}')
    codes = totalCodes
    k = 1
    pattern = initPattern
    hint = totalHints[(secretCode, pattern)]
    while True:
        codes = decreaseCodes(codes, pattern, hint)
        if verbose:
            print(f'{repr(pattern)} -> {repr(hint)} => {len(codes)} possibilites')
        if len(codes) == 1:
            guessCode = codes.pop()
            if secretCode != guessCode:
                raise Exception(f'ERREUR : {repr(secretCode)} => {repr(guessCode)} ??') # jamais exécuté
            if verbose:
                print(f'resultat en {k} propositions : {repr(guessCode)}')
            break
        k += 1
        pattern = nextPattern(codes)
        hint = totalHints[(secretCode, pattern)]
    return k

Test

In [8]:
_ = simulateKnuth((0,1,2,3))

secret : 🔴🟢🔵🟡
⚪🔴🔴⚪ -> ⚬        => 256 possibilites
🔵🟢🟢🔴 -> ● ⚬ ⚬    => 21 possibilites
🔴🟢🟡🔵 -> ● ● ⚬ ⚬  => 2 possibilites
🔵🟡🟢⚫ -> ⚬ ⚬ ⚬    => 1 possibilites
resultat en 4 propositions : 🔴🟢🔵🟡


L'exécution (longue : plus d'1/4 d'heure) de la cellule suivante calcule le maximum, pour tout code, du nombre de propositions nécessaire au calcul de ce code. Le résultat 4 (donc 5 si on compte la dernière étape de vérification) montre la validité de l'algorithme.

In [21]:
# Vérification de la validité (15 mns) 
verbose = False
for c in totalCodes:
    maxk = 0
    maxk = max(simulateKnuth(c, verbose = verbose), maxk)
    if verbose: print('')
if maxk == 4:
    print('Algorithme de Knuth prouvé')

Algorithme de Knuth prouvé


Les fonctions suivantes permettent à l'ordinateur décodeur de jouer contre un humain codeur

In [9]:
def initKnuth():

    global initPattern, currentCodes, currentPattern

    currentCodes = totalCodes
    currentPattern = initPattern
    print(f' motif proposé : {repr(initPattern)}')

def playKnuth(hint, verbose = True):

    global currentCodes, currentPattern
    
    currentCodes = decreaseCodes(currentCodes, currentPattern, hint)
    if verbose:
        print((f'{repr(currentPattern)} -> {repr(hint)} => {len(currentCodes)} possibilités'))
    if len(currentCodes) == 1:
        print(f'solution : {repr(currentCodes.pop())}')
    else:
        currentPattern = nextPattern(currentCodes)
        print(f' nouveau motif proposé : {repr(currentPattern)}')

Test

In [10]:
initKnuth()

 motif proposé : ⚪🔴🔴⚪


In [11]:
playKnuth((0,1))

⚪🔴🔴⚪ -> ⚬        => 256 possibilités
 nouveau motif proposé : 🔵🟢🟢🔴


In [12]:
playKnuth((1,2))

🔵🟢🟢🔴 -> ● ⚬ ⚬    => 21 possibilités
 nouveau motif proposé : 🔴🟢🟡🔵


In [13]:
playKnuth((2,2))

🔴🟢🟡🔵 -> ● ● ⚬ ⚬  => 2 possibilités
 nouveau motif proposé : 🔵🟡🟢⚫


In [14]:
playKnuth((0,3))

🔵🟡🟢⚫ -> ⚬ ⚬ ⚬    => 1 possibilités
solution : 🔴🟢🔵🟡


## L'algorithme de Greenwell
Greenwell propose les 6 motifs suivants

In [15]:
greenwell = (0,1,1,0), (1,2,4,3), (2,2,0,0), (3,4,1,3), (4,5,4,5), (5,5,3,2)
for pattern in greenwell: print(repr(pattern))

🔴🟢🟢🔴
🟢🔵⚫🟡
🔵🔵🔴🔴
🟡⚫🟢🟡
⚫⚪⚫⚪
⚪⚪🟡🔵


In [16]:

def playGreenwell(hints, verbose = True):
    codes = totalCodes
    for k, hint in enumerate(hints):
        pattern = greenwell[k]
        codes = decreaseCodes(codes, pattern, hint)
        if verbose:
            print(f'{repr(pattern)} -> {repr(hint)} => {len(codes)} possibilites')
        if len(codes) == 1:
            guessCode = codes.pop()
            if verbose:
                print(f'Résultat : {repr(guessCode)}')
            return guessCode
    print('INFORMATION INSUFFISANTE') # jamais exécuté

def simulateGreenwell(secretCode, verbose = True):
    if verbose:
        print(f'secret : {repr(secretCode)}')
    guessCode = playGreenwell([totalHints[(secretCode, greenwell[k])] for k in range(6)], verbose=verbose)
    if guessCode is None:
        return False
    if secretCode != guessCode:
        raise Exception(f'ERREUR : {secretCode} => {guessCode} ??')
    return True


In [17]:
_ = simulateGreenwell((0,1,2,3))

secret : 🔴🟢🔵🟡
🔴🟢🟢🔴 -> ● ●      => 114 possibilites
🟢🔵⚫🟡 -> ● ⚬ ⚬    => 10 possibilites
🔵🔵🔴🔴 -> ⚬ ⚬      => 2 possibilites
🟡⚫🟢🟡 -> ● ⚬      => 1 possibilites
Résultat : 🔴🟢🔵🟡


In [20]:
# Vérification de la validité
for c in totalCodes:
    if not simulateGreenwell(c, verbose = False):
        raise Exception(f"Le code {repr(c)} n'a pas été trouvé") # jamais exécuté
print('Algorithme de Greenwell prouvé')


Algorithme de Greenwell prouvé


Pour faire jouer le programme en lui donnant les  6 indications associées aux 6 motifs de `greenwell`

In [19]:
_ = playGreenwell([(2, 0), (1, 2), (0, 2), (1, 1), (0, 0), (0, 2)])

🔴🟢🟢🔴 -> ● ●      => 114 possibilites
🟢🔵⚫🟡 -> ● ⚬ ⚬    => 10 possibilites
🔵🔵🔴🔴 -> ⚬ ⚬      => 2 possibilites
🟡⚫🟢🟡 -> ● ⚬      => 1 possibilites
Résultat : 🔴🟢🔵🟡
